In [15]:
%%writefile in.langevin
# Sample LAMMPS input script for thermal conductivity of a metal: thermostatting 2 regions via fix langevin

# INITIALIZATION
variable        x equal 10
variable        y equal 10
variable        z equal 20

variable        T_low equal 200
variable        T_high equal 400
variable        T equal 300                 # Midway between T_low and T_high
variable        P equal 0.0

units           metal
dimension       3
boundary        p p p
atom_style      atomic
variable        element string "Al"         # Change this to the desired element

lattice fcc     4.05 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1 
region          box block 0 $x 0 $y 0 $z units lattice
create_box      1 box
create_atoms    1 box
                
region          hot_region block INF INF INF INF 0 1 units lattice
region          cold_region block  INF INF INF INF 10 11 units lattice
compute         T_hot all temp/region hot_region
compute         T_cold all temp/region cold_region

# INTERATOMIC POTENTIAL
pair_style      eam/alloy 
pair_coeff      * * CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy.txt ${element}

# EQUILIBRATION 1 - Equilibrate the system with a single temperature, T, and pressure, P
timestep        0.001
velocity        all create $T 87287
fix             1 all npt temp $T $T $(100.0*dt) iso $P $P $(1000.0*dt)
thermo          1000

shell if [ -d equil_1 ]; then rm -rf equil_1; fi
shell mkdir equil_1

dump            1 all custom 1000 equil_1/dump.equil_*.cfg id mass type x y z
run             20000

unfix           1
undump          1

# EQUILIBRATION 2 - Equilibrate the hot and cold regions
reset_timestep  0

fix             1 all nve

# tally will keep track of energy changes in the hot and cold regions
fix             hot_langevin all langevin ${T_high} ${T_high} $(100.0*dt) 59804 tally yes
fix_modify      hot_langevin temp T_hot

fix             cold_langevin all langevin ${T_low} ${T_low} $(100.0*dt) 287859 tally yes
fix_modify      cold_langevin temp T_cold

variable        step equal step
variable        tdiff equal c_T_hot-c_T_cold

thermo_style    custom step temp c_T_hot c_T_cold f_hot_langevin f_cold_langevin v_tdiff
thermo_modify   colname c_T_hot Temp_hot colname c_T_cold Temp_cold &
                colname f_hot_langevin E_hot colname f_cold_langevin E_cold &
                colname v_tdiff dTemp_step
thermo          1000

shell if [ -d equil_2 ]; then rm -rf equil_2; fi
shell mkdir equil_2
dump            2 all custom 1000 equil_2/dump.equil_*.cfg id mass type x y z

run             20000

undump          2

# THERMAL CONDUCTIVITY - Calculate the thermal conductivity previous equilibration
reset_timestep  0

# Every atom has an average kinetic energy of ⁠3/2 kB(eV/K) T(K) in thermal equilibrium.
compute         ke all ke/atom
variable        temp atom c_ke/1.5/8.61733326e-5

fix             1 all nve

# Reset langevin thermostats to zero energy accumulation
fix             hot_langevin all langevin ${T_high} ${T_high} $(100.0*dt) 59804 tally yes
fix_modify      hot_langevin temp T_hot

fix             cold_langevin all langevin ${T_low} ${T_low} $(100.0*dt) 287859 tally yes
fix_modify      cold_langevin temp T_cold

# Calculates the running average of the temperature difference between the hot and cold regions
# Input values every 10 steps. Running average is updated every 1000 steps, updated using the latest 100 samples.
fix             print_tdiff all print 10 "${step} ${tdiff}" file tdiff_values.dat screen no
fix             ave all ave/time 10 100 1000 v_tdiff ave running file ave_tdiff.dat   

thermo          1000
thermo_style    custom step temp c_T_hot c_T_cold f_hot_langevin f_cold_langevin v_tdiff f_ave
thermo_modify   colname c_T_hot Temp_hot colname c_T_cold Temp_cold &
                colname f_hot_langevin E_hot colname f_cold_langevin E_cold &
                colname v_tdiff dTemp_step colname f_ave dTemp_ave

# Bin the atoms into 20 layers along the z-axis
compute         layers all chunk/atom bin/1d z lower 0.05 units reduced

# Block average - Input values every 10 steps (average v_temp in each layer)
# In each layer, calculate the average temperature every 1000 steps, using the latest 100 samples.
# Coord1 in profile.langevin represents the center point of the bin
#fix            test all ave/chunk 10 1 10 layers v_temp file profile.langevin_test # For testing average v_temp in each layer
fix             2 all ave/chunk 10 100 1000 layers v_temp file profile.langevin

variable        start_time equal time

# f_hot_langevin and f_cold_langevin are the cumulative energies removed/added in the hot and cold regions, respectively.
# 0.5 * (abs(f_hot_langevin) + abs(f_cold_langevin)) is delta Q is the average of the total out/in energy for the 2 regions.
# A/2 since energy flux goes in 2 directions due to periodic z.
# lz/2.0 is the distance from the middle of the hot region to the middle of the cold region.
variable        kappa equal (0.5*(abs(f_hot_langevin)+abs(f_cold_langevin))/(time-${start_time})/(lx*ly)/2.0)*(lz/2.0)/f_ave
variable        kappa_wmk equal (0.5*(abs(f_hot_langevin)+abs(f_cold_langevin))/(time-${start_time})/(lx*ly)/2.0)*(lz/2.0)/f_ave*1.60218e-19/1e-12/1e-10
        
shell if [ -d kappa ]; then rm -rf kappa; fi
shell mkdir kappa
dump            3 all custom 100 kappa/dump.kappa_*.cfg id mass type x y z v_temp

run             20000
print           "time: $(time)"
print           "Final running average thermal conductivity: $(v_kappa)"
print           "Final running average thermal conductivity: $(v_kappa_wmk) W/mK"



Overwriting in.langevin


Run on LAMMPS on your personal computer or a cluster

In [5]:
!lmp < in.langevin

LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4.05 4.05 4.05
Created orthogonal box = (0 0 0) to (40.5 40.5 81)
  1 by 1 by 1 MPI processor grid
Created 8000 atoms
  using lattice units in orthogonal box = (0 0 0) to (40.5 40.5 81)
  create_atoms CPU = 0.001 seconds
Reading eam/alloy potential file CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy.txt with DATE: 2018-03-30
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 9.8256185
  ghost atom cutoff = 9.8256185
  binsize = 4.9128093, bins = 9 9 17
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d
      bin: standard
Setting up Verlet run ...
 

In [156]:
# Read the profile.langevin file to plot the temperature profile
with open('profile.langevin', 'r') as f:
    lines = f.readlines()

lines = [line.split() for line in lines]

timestep = []
coord1_list = []
coord1_chunk = []
v_temp_list = []
v_temp_chunk = []
for line in lines:
    if line[0] == '#':
        continue
    elif len(line) == 3:
        timestep.append(int(line[0]))
        number_of_chunks = int(line[1])
    else:
        if len(coord1_chunk) < number_of_chunks:
            coord1_chunk.append(float(line[1]))
            v_temp_chunk.append(float(line[3]))

        elif len(coord1_chunk) == number_of_chunks:
            coord1_list.append([coord1_chunk])
            coord1_chunk = []
            coord1_chunk.append(float(line[1]))
            
            v_temp_list.append([v_temp_chunk])
            v_temp_chunk = []
            v_temp_chunk.append(float(line[3]))
            
if coord1_chunk:
    coord1_list.append([coord1_chunk])
if v_temp_chunk:
    v_temp_list.append([v_temp_chunk])
    
mean_v_temps = [np.mean(v_temp[0]) for v_temp in v_temp_list]


In [157]:
import plotly.graph_objects as go

fig = go.Figure()
for i in range(len(coord1_list)):
    fig.add_trace(go.Scatter(x=coord1_list[i][0], y=v_temp_list[i][0], mode='lines', name=timestep[i]))

# Formatting
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(width=900, height=600)

fig.update_layout(plot_bgcolor='white')

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

fig.update_xaxes(title_text='z', title_font=dict(size=18))
fig.update_xaxes(tickfont=dict(size=14))
fig.update_yaxes(title_text='Temperature', title_font=dict(size=18))
fig.update_yaxes(tickfont=dict(size=14))

fig.update_xaxes(range=[0, 1], dtick=0.1)
fig.update_yaxes(range=[1, 1.7], dtick=0.1)

fig.add_shape(type='line', x0=0, y0=0, x1=0, y1=2, line=dict(color='red', width=1))
fig.add_shape(type='line', x0=0.05, y0=0, x1=0.05, y1=2, line=dict(color='red', width=1))
fig.add_shape(type='line', x0=0.5, y0=0, x1=0.5, y1=2, line=dict(color='blue', width=1))
fig.add_shape(type='line', x0=0.55, y0=0, x1=0.55, y1=2, line=dict(color='blue', width=1))
fig.show()
